In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras.layers import TextVectorization, Embedding
from tensorflow.keras import Sequential

In [3]:
batch_size = 1024
seed = 39
sequence_length = 25
vocab_size = 4096
embedding_dim = 128

train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "dataset", batch_size = batch_size, validation_split = 0.2, 
    subset = 'training', seed = seed
)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "dataset", batch_size = batch_size, validation_split = 0.2, 
    subset = 'validation', seed = seed
)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size =  AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size =  AUTOTUNE)



vectorize_layer = TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int', 
    output_sequence_length = sequence_length)

text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

Found 230710 files belonging to 2 classes.
Using 184568 files for training.
Found 230710 files belonging to 2 classes.
Using 46142 files for validation.


In [4]:




class EmbeddingInitializer(tf.keras.initializers.Initializer):
    def __call__(self, shape=None, dtype=None, **kwargs):
        weights = tf.convert_to_tensor(np.load('w2vVectors.npy'))
        return weights

embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            embeddings_initializer = EmbeddingInitializer(),
                            mask_zero= True)




model = Sequential([
    vectorize_layer,
    embedding_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

model.fit(train_ds, epochs=25,
                    validation_data=val_ds,
                    validation_steps=30, 
                    callbacks = tensorboard_callback)

Epoch 1/25
181/181 [==============================] - 307s 2s/step - loss: 0.6894 - accuracy: 0.5009 - val_loss: 0.6866 - val_accuracy: 0.4924
Epoch 2/25
181/181 [==============================] - 235s 1s/step - loss: 0.6835 - accuracy: 0.5036 - val_loss: 0.6822 - val_accuracy: 0.5030
Epoch 3/25
181/181 [==============================] - 225s 1s/step - loss: 0.6786 - accuracy: 0.5137 - val_loss: 0.6790 - val_accuracy: 0.5146
Epoch 4/25
181/181 [==============================] - 221s 1s/step - loss: 0.6740 - accuracy: 0.5233 - val_loss: 0.6770 - val_accuracy: 0.5222
Epoch 5/25
181/181 [==============================] - 228s 1s/step - loss: 0.6695 - accuracy: 0.5316 - val_loss: 0.6752 - val_accuracy: 0.5261
Epoch 6/25
181/181 [==============================] - 221s 1s/step - loss: 0.6649 - accuracy: 0.5405 - val_loss: 0.6719 - val_accuracy: 0.5315
Epoch 7/25
181/181 [==============================] - 233s 1s/step - loss: 0.6603 - accuracy: 0.5492 - val_loss: 0.6706 - val_accuracy: 0.5347

In [5]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


hi this is a VCS test